In [1]:
import sys
sys.path.append('..')
import numpy as np
import torch
from tqdm import trange
from IPython.display import Video
from utils.video import write_video, transpose_and_clip
from utils.vqvae import Decoder, CompressorConfig

In [2]:
# load model
config = CompressorConfig()
with torch.device('meta'):
  decoder = Decoder(config)
decoder.load_state_dict_from_url('https://huggingface.co/commaai/commavq-gpt2m/resolve/main/decoder_pytorch_model.bin', assign=True)
decoder = decoder.eval().to(device='cuda')

In [3]:
# load tokens
tokens = np.load("../examples/tokens.npy").astype(np.int64)
tokens = torch.from_numpy(tokens).to(device='cuda')

In [4]:
# decoding loop
decoded_video = []
with torch.no_grad():
  for i in trange(len(tokens)):
    decoded = decoder(tokens[i][None])
    decoded_video.append(decoded)
decoded_video = torch.cat(decoded_video, dim=0).cpu().numpy()

100%|██████████| 1200/1200 [00:31<00:00, 37.69it/s]


In [5]:
# transpose and format video
decoded_video = transpose_and_clip(decoded_video)

In [6]:
# save video
save_dst = 'commavq/tmp/decoded.mp4'
write_video(decoded_video, save_dst, fps=20)
Video(save_dst, embed=True, width=700)